In [1]:
#import tensorflow as tf
import os


import numpy as np
import pandas as pd
import re
import scipy as sp
from scipy.sparse import hstack, vstack
from sklearn.metrics import accuracy_score
from sklearn.metrics import log_loss
from sklearn.metrics import roc_auc_score
import lightgbm as lgb
from sklearn.model_selection import KFold

import sklearn
from sklearn.svm import LinearSVC
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.ensemble import BaggingClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.naive_bayes import BernoulliNB

import pickle
from sklearn.externals import joblib
from sklearn.model_selection import train_test_split

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score
from sklearn.pipeline import make_union
import sys
sys.executable

'/home/yichen/anaconda3/bin/python'

In [2]:
def save_variable_pickle(var, name, model = False):
    filename = name + '.pickle'
    if model:
        joblib.dump(var, filename = filename)
    else:  
        with open(filename, 'wb') as f:
            pickle.dump(var, f, pickle.HIGHEST_PROTOCOL)

def load_variable_pickle(name, model = False):
    filename = name
    if model:
        return joblib.load(filename = filename)
    else:
        with open(filename, 'rb') as f:
            return pickle.load(f)
        

In [3]:
def choose_system(os):
    if os == 'mac':
        train_path = '/Users/yichen/Desktop/toxic/input/train.csv'
        test_path = '/Users/yichen/Desktop/toxic/input/test.csv'
    elif os == 'linux':
        train_path = '/home/yichen/Desktop/toxic/input/train.csv'
        test_path = '/home/yichen/Desktop/toxic/input/test.csv'
    elif os == 'aws':
        train_path = 'train.csv'
        test_path = 'test.csv'
    
    train = pd.read_csv(train_path).fillna(' ')
    test = pd.read_csv(test_path).fillna(' ')
    return train, test
    
    

In [4]:
def check_accuracy(true, estimate):
    acc = accuracy_score(true, estimate)
    logloss = log_loss(true, estimate)
    auc = roc_auc_score(true, esimate)
    print("accuracy score: {}, logloss: {}, auc: {}".format(acc, logloss, auc))

In [5]:
train, test = choose_system('linux')
train_text = train['comment_text']
test_text = test['comment_text']
all_text = pd.concat([train_text, test_text])
class_names = ['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']
test.shape

(153164, 2)

In [6]:
repl = {
    "&lt;3": " good ", ":d": " good ",
    ":dd": " good ",
    ":p": " good ",
    "8)": " good ",
    ":-)": " good ",
    ":)": " good ",
    ";)": " good ",
    "(-:": " good ",
    "(:": " good ",
    "yay!": " good ",
    "yay": " good ",
    "yaay": " good ",
    "yaaay": " good ",
    "yaaaay": " good ",
    "yaaaaay": " good ",
    ":/": " bad ",
    ":&gt;": " sad ",
    ":')": " sad ",
    ":-(": " bad ",
    ":(": " bad ",
    ":s": " bad ",
    ":-s": " bad ",
    "&lt;3": " heart ",
    ":d": " smile ",
    ":p": " smile ",
    ":dd": " smile ",
    "8)": " smile ",
    ":-)": " smile ",
    ":)": " smile ",
    ";)": " smile ",
    "(-:": " smile ",
    "(:": " smile ",
    ":/": " worry ",
    ":&gt;": " angry ",
    ":')": " sad ",
    ":-(": " sad ",
    ":(": " sad ",
    ":s": " sad ",
    ":-s": " sad ",
    r"\br\b": "are",
    r"\bu\b": "you",
    r"\bhaha\b": "ha",
    r"\bhahaha\b": "ha",
    r"\bdon't\b": "do not",
    r"\bdoesn't\b": "does not",
    r"\bdidn't\b": "did not",
    r"\bhasn't\b": "has not",
    r"\bhaven't\b": "have not",
    r"\bhadn't\b": "had not",
    r"\bwon't\b": "will not",
    r"\bwouldn't\b": "would not",
    r"\bcan't\b": "can not",
    r"\bcannot\b": "can not",
    r"\bi'm\b": "i am",
    "m": "am",
    "r": "are",
    "u": "you",
    "haha": "ha",
    "hahaha": "ha",
    "don't": "do not",
    "doesn't": "does not",
    "didn't": "did not",
    "hasn't": "has not",
    "haven't": "have not",
    "hadn't": "had not",
    "won't": "will not",
    "wouldn't": "would not",
    "can't": "can not",
    "cannot": "can not",
    "i'm": "i am",
    "m": "am",
    "i'll" : "i will",
    "its" : "it is",
    "it's" : "it is",
    "'s" : " is",
    "that's" : "that is",
    "weren't" : "were not",
}

In [7]:
keys = [i for i in repl.keys()]

new_train_data = []
new_test_data = []
ltr = train["comment_text"].tolist()
keys = [i for i in repl.keys()]

new_train_data = []
new_test_data = []
ltr = train["comment_text"].tolist()
lte = test["comment_text"].tolist()
for i in ltr:
    arr = str(i).split()
    xx = ""
    for j in arr:
        j = str(j).lower()
        if j[:4] == 'http' or j[:3] == 'www':
            continue
        if j in keys:
            # print("inn")
            j = repl[j]
        xx += j + " "
    new_train_data.append(xx)
for i in lte:
    arr = str(i).split()
    xx = ""
    for j in arr:
        j = str(j).lower()
        if j[:4] == 'http' or j[:3] == 'www':
            continue
        if j in keys:
            # print("inn")
            j = repl[j]
        xx += j + " "
    new_test_data.append(xx)
train["new_comment_text"] = new_train_data
test["new_comment_text"] = new_test_data

trate = train["new_comment_text"].tolist()
tete = test["new_comment_text"].tolist()
for i, c in enumerate(trate):
    trate[i] = re.sub('[^a-zA-Z ?!]+', '', str(trate[i]).lower())
for i, c in enumerate(tete):
    tete[i] = re.sub('[^a-zA-Z ?!]+', '', tete[i])
train["comment_text"] = trate
test["comment_text"] = tete
del trate, tete
train.drop(["new_comment_text"], axis=1, inplace=True)
test.drop(["new_comment_text"], axis=1, inplace=True)

train_text = train['comment_text']
test_text = test['comment_text']
all_text = pd.concat([train_text, test_text])

In [8]:
all_text.shape

(312735,)

In [ ]:
word_vectorizer = TfidfVectorizer(
    strip_accents='unicode',
    analyzer='word',
    min_df = 2,
    stop_words = 'english',
    token_pattern=r'\w{1,}',
    ngram_range=(1, 3),
    max_features=30000
    )
    

char_vectorizer = TfidfVectorizer(
    strip_accents='unicode',
    analyzer='char',
    min_df = 2,
    ngram_range=(2, 6),
    max_features=30000)

vectorizer = make_union(word_vectorizer, char_vectorizer)

In [10]:
vectorizer.fit(all_text)
train_features = vectorizer.transform(train_text)
train_features.shape

(159571, 60000)

In [11]:
test_features = vectorizer.transform(test_text)

In [12]:
test_features.shape

(153164, 60000)

In [13]:
import pickle
from sklearn.externals import joblib
save_variable_pickle(train_features, 'train_features_save')
save_variable_pickle(test_features, 'test_features_save')

In [9]:
import pickle
from sklearn.externals import joblib
test_features = load_variable_pickle('test_features_save.pickle')
train_features = load_variable_pickle('train_features_save.pickle')
y_fold_renew = load_variable_pickle('y_fold_renew.pickle')
#clf = load_variable_pickle('_logistic_model5.pickle')

In [10]:
print(test_features.shape, train_features.shape, y_fold_renew.shape)


(153164, 60000) (159571, 60000) (159571, 6)


In [11]:
def split_K_fold(train, target, K, model):
    acc = []
    kf = KFold(n_splits=K, random_state=1023)
    count = 0
    for train_index, test_index in kf.split(train, target):
        X_train, X_test = train[train_index], train[test_index]
        y_train, y_test = target[train_index], target[test_index]
        #print("x_train: {}, x_test: {}, y_train: {}, y_test: {}". format(X_train.shape, X_test.shape, y_train.shape, y_test.shape))
        model.fit(X_train, y_train)
        if count == 0:
            pred = model.predict(X_test)
            y_fold_valid = y_test
            acc.append(roc_auc_score(pred, y_test))
        else:
            pred1 = model.predict(X_test)
            pred = np.concatenate((pred, pred1), axis = 0)
            #y_fold_valid = np.concatenate((y_fold_valid, y_test))
            #print(roc_auc_score(pred1, y_test))
            acc.append(roc_auc_score(pred1, y_test))
        count = count + 1
        #print("y_fold_valid.shape: ", y_fold_validation.shape, "pred.shape: ", pred.shape)
    print("average roc_auc_score for model: {} is: {}  for model type: {} ".format(str(type(model)), np.mean(acc), type(model)))
    #y_fold_name = 'y_fold_valid' + str(type(model))
    #save_variable_pickle(y_fold_valid, y_fold_name)
    return pred

In [12]:
def ready_for_leveltwo(models, train_data, test_data, target,y_fold, meta_model):
    #os_load_file_path = '/home/yichen/Desktop/toxic/base_model/'
    class_counter = 0
    submission = pd.DataFrame.from_dict({'id': test['id']})
    for class_name in class_names:
        os_load_file_path = '/home/yichen/Desktop/toxic/base_model/'
        os_load_submission_path = '/home/yichen/Desktop/toxic/submission/'
        os_load_submission_path = os_load_file_path + str(class_name) + str(type(meta_model))
        os_load_file_path = os_load_file_path + str(class_name) + '/'
        print(os_load_file_path)
        count = 0
        train_target = target[class_name]
        for model in models:
            name = os_load_file_path + '_' + class_name + '_' + str(type(model)) + 'cv'+ '_'+ '.pickle'
            if count == 0:
                if os.path.exists(name):
                    with open(name, 'rb') as f:
                        meta_train = pickle.load(f)
                    print("old prediction stored: {}, shape: {}".format(type(model), meta_train.shape))
                else:
                    meta_train = split_K_fold(train_data, train_target, 5, model)
                   # cv_split(train = train_data, target=train_target, K = 5, random_state=1023, model = model)
                    with open(name, 'wb') as f:
                        pickle.dump(meta_train, f, pickle.HIGHEST_PROTOCOL)
                count = count + 1
            else:
                if os.path.exists(name):
                    with open(name, 'rb') as f:
                        temp = pickle.load(f)
                    print("old prediction stored: {}, shape: {}".format(type(model), temp.shape))
                else:
                    temp = split_K_fold(train_data, train_target, 5, model)
                    with open(name, 'wb') as f:
                        pickle.dump(temp, f, pickle.HIGHEST_PROTOCOL)
                meta_train = np.column_stack((meta_train, temp))
                count = count + 1
                
        count = 0                
        print("cv finished")
        for model in models:
            #print("count", count)
            name = os_load_file_path + '_' + class_name + '_' + str(type(model)) + 'base_lvl' + '_' + '.pickle'
            #print(train_target.shape)
            
            if count == 0:
                if os.path.exists(name):
                    with open(name, 'rb') as f:
                        meta_test = pickle.load(f)
                        print("meta_test:",meta_test.shape)
                    print("old prediction stored: {}, shape: {}".format(type(model), meta_train.shape))
                else:
                    model.fit(train_data, train_target)
                    meta_test = model.predict(test_data)
                    print(meta_test.shape)
                    with open(name, 'wb') as f:
                        pickle.dump(meta_test, f, pickle.HIGHEST_PROTOCOL)
                    print("model finish predication: {}".format(str(type(model))))
                count = count + 1
            else:
                if os.path.exists(name):
                    with open(name, 'rb') as f:
                        temp = pickle.load(f)
                    print("old prediction stored: {}, shape: {}".format(type(model), meta_train.shape))
                    print(temp.shape)
                else:
                    model.fit(train_data, train_target)
                    temp = model.predict(test_data)
                    with open(name, 'wb') as f:
                        pickle.dump(temp, f, pickle.HIGHEST_PROTOCOL)
                    print("model finish predication: {}".format(str(type(model))))
                    print(temp.shape)
                print("meta_test: ", meta_test.shape, "model: ", str(type(model)))
                meta_test = np.column_stack((meta_test, temp))
                #print("meta_test: ",meta_test.shape)
                count = count + 1
    
            
        
        meta_train_valid = y_fold[:, class_counter]
        #print("meta_train: {}, y_fold: {}, meta_test: {}".format(meta_train.shape, meta_train_valid.shape, meta_test.shape))
        if os.path.exists(os_load_submission_path):
            with open(name, 'rb') as f:
                        a = pickle.load(f)
        
        else:
            meta_model.fit(meta_train, meta_train_valid) 
            a = meta_model.predict(meta_test)
            with open(name, 'wb') as f:
                        pickle.dump(os_load_submission_path, f, pickle.HIGHEST_PROTOCOL)
        
        print(a.shape)
        submission[class_name] = a
        class_counter = class_counter + 1
        print("round: ")
    submission.to_csv('submission_final.csv', index=False)
    return submission

In [13]:
from sklearn import svm

ada = AdaBoostClassifier(n_estimators=100)

rfc = RandomForestClassifier(n_estimators=200, min_samples_leaf= 200, random_state=1023, verbose = True)
nusvc = svm.NuSVC(nu=0.01,random_state = 1023, verbose = True)
#rfc= RandomForestClassifier(n_estimators= 50, random_state=11, verbose = True)
#submission[class_name] = classifier.predict_proba(test_features)[:, 1]
MNB = MultinomialNB()
gdc = GradientBoostingClassifier(n_estimators=400, 
 learning_rate = 0.1, max_depth=5, random_state=1023, verbose = True)
gbm_tune = lgb.LGBMClassifier(objective = 'binary',
                        metric = 'auc', max_depth=12, min_data_in_leaf = 50, num_iterations = 100, 
                              learning_rate = 0.1
                              , device = 'gpu')
gbm = lgb.LGBMClassifier(objective = 'binary',
                        metric = 'auc',
                        num_leaves=31,
                        learning_rate=0.05,
                        n_estimators=500, device = 'gpu')
etc = ExtraTreesClassifier(n_estimators=200, min_samples_leaf= 200, random_state=1023, verbose = True)
log_model = LogisticRegression(solver='sag')
svc_model = LinearSVC(random_state=1023)

model_set1 = []
model_set1.append(svc_model)
model_set1.append(log_model)
model_set1.append(MNB)
model_set1.append(etc)
#model_set1.append(ada)
model_set1.append(gbm_tune)
model_set1.append(rfc)
#model_set1.append(gdc)
model_set1.append(nusvc)
#model_set1.append(neigh2)
#model_set1.append(neigh4)
#model_set1.append(neigh8)
#model_set1.append(neigh16)
#model_set1.append(neigh32)
#model_set1.append(neigh64)

In [14]:
meta_train1 = ready_for_leveltwo(model_set1, train_data=train_features, test_data=test_features, target=train,y_fold=y_fold_renew, meta_model=gbm)
meta_train1.shape

/home/yichen/Desktop/toxic/base_model/toxic/
old prediction stored: <class 'sklearn.svm.classes.LinearSVC'>, shape: (159571,)
old prediction stored: <class 'sklearn.linear_model.logistic.LogisticRegression'>, shape: (159571,)
old prediction stored: <class 'sklearn.naive_bayes.MultinomialNB'>, shape: (159571,)
old prediction stored: <class 'sklearn.ensemble.forest.ExtraTreesClassifier'>, shape: (159571,)
old prediction stored: <class 'lightgbm.sklearn.LGBMClassifier'>, shape: (159571,)
old prediction stored: <class 'sklearn.ensemble.forest.RandomForestClassifier'>, shape: (159571,)
old prediction stored: <class 'sklearn.svm.classes.NuSVC'>, shape: (159571,)
cv finished
meta_test: (153164,)
old prediction stored: <class 'sklearn.svm.classes.LinearSVC'>, shape: (159571, 7)
old prediction stored: <class 'sklearn.linear_model.logistic.LogisticRegression'>, shape: (159571, 7)
(153164,)
meta_test:  (153164,) model:  <class 'sklearn.linear_model.logistic.LogisticRegression'>
old prediction sto

AttributeError: 'str' object has no attribute 'shape'

In [ ]:
model_set1.append(neigh2)
model_set1.append(neigh4)
model_set1.append(neigh8)
model_set1.append(neigh16)
model_set1.append(neigh32)
meta_train2 = ready_for_leveltwo(model_set1, train_features, test_features, y_fold_validation, meta_model=gbm)

In [24]:
def cv_split(train, target, K, random_state ,model):
    acc = []
    row = train.shape[0]
    if row != target.shape[0]:
        raise ValueError("At least one array required as input")
    num_per_fold = np.int(np.ceil(train.shape[0]/K))
    x_train, x_fold, y_train, y_fold1 = train_test_split(train, target, test_size = num_per_fold, random_state = random_state)
    model.fit(x_train, y_train)
    pred = model.predict(x_fold)
    acc.append(roc_auc_score(y_fold1, pred))
    #print("accuracy is {}".format(accuracy_score(y_fold1, pred)))
    i = 1
    if train.size != row:
        while i < K:
            if i != K - 1:
                x_fold = train[num_per_fold * i: num_per_fold * (i + 1), :]
                y_fold = target[num_per_fold * i: num_per_fold * (i + 1)]
                x_train = vstack([train[0: num_per_fold * i, : ], train[num_per_fold * (i + 1): , :]])
                y_train = pd.concat([target[0 : num_per_fold * i], target[num_per_fold * (i + 1): ]])
                #print("x_train: {}, y_train: {}, x_fold: {}, y_fold: {}".format(x_train.shape, y_train.shape, x_fold.shape, y_fold.shape))
            elif i == K-1:
                x_fold = train[num_per_fold * i: , :]
                y_fold = target[num_per_fold * i: ]
                x_train = train[0:num_per_fold * i, :]
                y_train = target[0:num_per_fold * i]
                
            
            y_fold1 = pd.concat([y_fold1[:], y_fold[:]])
            model.fit(x_train, y_train)
            pred1 = model.predict(x_fold)
            acc.append(roc_auc_score(y_fold, pred1))
            #rint("accuracy is {}".format(accuracy_score(y_fold, pred1)))
            pred = np.concatenate((pred, pred1), axis = 0)
            i = i + 1
    print("average roc_auc_score is: {}  for model type: {} ".format(np.mean(acc), type(model)))
    return pred# pred_all_testep

In [15]:
def single_class_cv_mutiple_time(model):
    result = np.arange(159571*6).reshape(159571,6)
    #all_test_result = np.arange(153164*6).reshape(153164,6)
    col = 0
    for class_name in class_names:
        train_target = train[class_name]
        test = test_features
        pred1, overall_test_result= cv_split(train = train_features, target = train_target, K = 5, random_state=1023, model=model)
        result[:, col] = pred1
        all_test_result[:, col] = overall_test_result
        #y_fold_result[:, col] = y_fold
        col = col + 1
        print("pred shape: {} and overall_test shape: {} ".format(pred1.shape, all_test_result.shape))
    
    return result, all_test_result


In [16]:
def param_tune_all_class(model, train, train_t):
    score_auc = []
    for class_name in class_names:
        train_target = train_t[class_name]
        cv_score = np.mean(cross_val_score(model, train, train_target, cv=3, scoring='roc_auc'))
        scores_auc.append(np.abs(cv_score))
        print('auc CV score for class {} is {}'.format(class_name, np.abs(cv_score)))
    print('overall CV score for class is {}'.format(np.mean(score_auc)))

def param_tune_one_class(model, train, train_target):
    cv_score = np.mean(cross_val_score(model, train, train_target, cv=3, scoring='roc_auc'))
    print('auc CV score for class is {}'.format(np.abs(cv_score)))
    #print('overall CV score for class is {}'.format(np.mean(score_auc)))

In [17]:
import sklearn

#print('The nltk version is {}.'.format(nltk.__version__))
print('The scikit-learn version is {}.'.format(sklearn.__version__))

The scikit-learn version is 0.19.1.


In [27]:
gdc_max_depth = [3, 5, 7, 10]
#for max_d in gdc_max_depth:
gdc = GradientBoostingClassifier(n_estimators=150,# validation_fraction=0.2,
     learning_rate = 0.15, max_depth=5, random_state=1023, verbose = True)
param_tune_one_class(gdc, train_features, train['toxic'])

      Iter       Train Loss   Remaining Time 
         1           0.5312          114.70m
         2           0.4968          110.61m
         3           0.4737          109.85m
         4           0.4500          110.46m
         5           0.4324          109.84m
         6           0.4175          109.34m
         7           0.4033          108.66m
         8           0.3902          107.71m
         9           0.3824          107.06m
        10           0.3750          106.17m
        20           0.3124          103.25m


KeyboardInterrupt: 

In [30]:
rfc5 = RandomForestClassifier(n_estimators=200, min_samples_leaf= 200, random_state=1023, verbose = True)
param_tune_one_class(rfc5, train_features, train['toxic'])

[Parallel(n_jobs=1)]: Done 200 out of 200 | elapsed:  2.3min finished
[Parallel(n_jobs=1)]: Done 200 out of 200 | elapsed:   23.2s finished
[Parallel(n_jobs=1)]: Done 200 out of 200 | elapsed:  2.2min finished
[Parallel(n_jobs=1)]: Done 200 out of 200 | elapsed:   28.3s finished
[Parallel(n_jobs=1)]: Done 200 out of 200 | elapsed:  2.2min finished


auc CV score for class is 0.9260250228550927


[Parallel(n_jobs=1)]: Done 200 out of 200 | elapsed:   30.6s finished


In [31]:
rfc5 = RandomForestClassifier(n_estimators=200, max_depth = 7, min_samples_leaf= 400, random_state=1023, verbose = True)
param_tune_one_class(rfc5, train_features, train['toxic'])

[Parallel(n_jobs=1)]: Done 200 out of 200 | elapsed:  1.3min finished
[Parallel(n_jobs=1)]: Done 200 out of 200 | elapsed:   28.5s finished
[Parallel(n_jobs=1)]: Done 200 out of 200 | elapsed:  1.3min finished
[Parallel(n_jobs=1)]: Done 200 out of 200 | elapsed:   29.8s finished
[Parallel(n_jobs=1)]: Done 200 out of 200 | elapsed:  1.3min finished


auc CV score for class is 0.9050253348431814


[Parallel(n_jobs=1)]: Done 200 out of 200 | elapsed:   19.8s finished


In [19]:
gbm_tune = lgb.LGBMClassifier(objective = 'binary',
                        metric = 'auc', max_depth=12, min_data_in_leaf = 50, num_iterations = 100, 
                              learning_rate = 0.1
                              , device = 'gpu')
param_tune_one_class(gbm_tune, train_features, train['toxic'])

/home/yichen/anaconda3/lib/python3.6/site-packages/lightgbm/engine.py:99: UserWarning: Found `num_iterations` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))
/home/yichen/anaconda3/lib/python3.6/site-packages/lightgbm/engine.py:99: UserWarning: Found `num_iterations` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))
/home/yichen/anaconda3/lib/python3.6/site-packages/lightgbm/engine.py:99: UserWarning: Found `num_iterations` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))


auc CV score for class is 0.9679447925002233


In [ ]:
rfc5 = RandomForestClassifier(n_estimators=300, 
                              max_depth = 7, random_state=1023, verbose = True)
param_tune_one_class(rfc5, train_features, train['toxic'])

In [ ]:
rfc5 = RandomForestClassifier(n_estimators= 400,max_depth = 7, random_state=1023, verbose = True)
param_tune_one_class(rfc5, train_features, train['toxic'])

In [ ]:
from sklearn.ensemble import ExtraTreesClassifier
etc5 = ExtraTreesClassifier(n_estimators=200, min_samples_leaf= 200, random_state=1023, verbose = True)
param_tune_one_class(etc5, train_features, train['toxic'])

In [ ]:
etc5 = ExtraTreesClassifier(n_estimators=200, min_samples_leaf= 400, random_state=1023, verbose = True)
param_tune_one_class(etc5, train_features, train['toxic'])

In [ ]:
etc5 = ExtraTreesClassifier(n_estimators=130, min_samples_leaf= 400, random_state=1023, verbose = True)
param_tune_one_class(etc5, train_features, train['toxic'])

In [ ]:
etc5 = ExtraTreesClassifier(n_estimators=400, min_samples_leaf= 200, random_state=1023, verbose = True)
param_tune_one_class(etc5, train_features, train['toxic'])

In [ ]:
etc5 = ExtraTreesClassifier(n_estimators=600, min_samples_leaf= 200, random_state=1023, verbose = True)
param_tune_one_class(etc5, train_features, train['toxic'])

In [ ]:
gdc = GradientBoostingClassifier(n_estimators=200,# validation_fraction=0.2,
     learning_rate = 0.15, max_depth=5, random_state=1023, verbose = True)
param_tune_one_class(gdc, train_features, train['toxic'])

In [ ]:
gdc = GradientBoostingClassifier(n_estimators=200,# validation_fraction=0.2,
     learning_rate = 0.15, max_depth=7, random_state=1023, verbose = True)
param_tune_one_class(gdc, train_features, train['toxic'])

In [ ]:
gdc = GradientBoostingClassifier(n_estimators=200,# validation_fraction=0.2,
     learning_rate = 0.15, max_depth=10, random_state=1023, verbose = True)
param_tune_one_class(gdc, train_features, train['toxic'])

In [92]:
param_tune_one_class(nusvc, train_features, train['toxic'])

[LibSVM][LibSVM][LibSVM]auc CV score for class is 0.9179922166947202


In [139]:
scores_auc = []
submission = pd.DataFrame.from_dict({'id': test['id']})
#classifier = LogisticRegression()
#for class_name in class_names:
train_target = train['toxic']
    #print(train_target.shape)
classifier = LogisticRegression(solver='sag')

    #cv_score = np.mean(cross_val_score(
     #   classifier, train_features, train_target, cv=3, scoring='neg_log_loss'))
    #scores_auc.append(np.abs(cv_score))
    #print('auc CV score for class {} is {}'.format(class_name, np.abs(cv_score)))

classifier.fit(train_features, train_target)
a = classifier.predict_proba(test_features)
print(a.shape)
a
submission[class_name] = a[:, 1]
    #save_varible_pickle(classifier, _logistic, model = True)
#print('Total auc CV score is {}'.format(np.mean(scores_auc)))

(153164, 2)


In [18]:
submission.to_csv('submission.csv', index=False)

In [ ]:
def cv_split(train, target, K, random_state ,model):
    acc = []
    row = train.shape[0]
    if row != target.shape[0]:
        raise ValueError("At least one array required as input")
    num_per_fold = np.int(np.ceil(train.shape[0]/K))
    x_train, x_fold, y_train, y_fold1 = train_test_split(train, target, test_size = num_per_fold, random_state = random_state)
    model.fit(x_train, y_train)
    pred = model.predict(x_fold)
    acc.append(roc_auc_score(y_fold1, pred))
    #print("accuracy is {}".format(accuracy_score(y_fold1, pred)))
    i = 1
    if train.size != row:
        while i < K:
            if i != K - 1:
                x_fold = train[num_per_fold * i: num_per_fold * (i + 1), :]
                y_fold = target[num_per_fold * i: num_per_fold * (i + 1)]
                x_train = vstack([train[0: num_per_fold * i, : ], train[num_per_fold * (i + 1): , :]])
                y_train = pd.concat([target[0 : num_per_fold * i], target[num_per_fold * (i + 1): ]])
                #print("x_train: {}, y_train: {}, x_fold: {}, y_fold: {}".format(x_train.shape, y_train.shape, x_fold.shape, y_fold.shape))
            elif i == K-1:
                x_fold = train[num_per_fold * i: , :]
                y_fold = target[num_per_fold * i: ]
                x_train = train[0:num_per_fold * i, :]
                y_train = target[0:num_per_fold * i]
                
            
            y_fold1 = pd.concat([y_fold1[:], y_fold[:]])
            model.fit(x_train, y_train)
            pred1 = model.predict(x_fold)
            acc.append(roc_auc_score(y_fold, pred1))
            #rint("accuracy is {}".format(accuracy_score(y_fold, pred1)))
            pred = np.concatenate((pred, pred1), axis = 0)
            i = i + 1
    print("average roc_auc_score is: {}  for model type: {} ".format(np.mean(acc), type(model)))
    return pred# pred_all_testep